In [ ]:
# Lets read in aggregated_df.csv and run a linear regression on it
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

In [ ]:
df = pd.read_csv('data/aggregated_df.csv')
# Rename columns by replacing spaces with underscores
df.columns = df.columns.str.replace(' ', '_').str.lower()
print(df.columns)

In [ ]:
# Function to add model results to a list
def add_model_to_results(data, predictors, outcome, results_list):
    formula = "{} ~ {}".format(outcome, ' + '.join(predictors))
    model = smf.ols(formula, data=data).fit()
    model_summary = {
        'Model': ' + '.join(predictors),
        'AIC': model.aic,
        'BIC': model.bic,
        'R-squared': model.rsquared,
        'Adj. R-squared': model.rsquared_adj,
        'Coefficients': model.params.to_dict()
    }
    results_list.append(model_summary)

# Function to add predictors based on correlation
def add_predictors_by_correlation(df, predictors, outcome):
    correlation_with_outcome = df[predictors].corrwith(df[outcome]).abs().sort_values(ascending=False)
    included = []
    results_list = []
    for predictor in correlation_with_outcome.index:
        included.append(predictor)
        add_model_to_results(df, included, outcome, results_list)
    return included, results_list

# Define your predictors and outcome variable
predictors = ['smart_alarm', 'drinks', 'fastfood', 'sports', 'food_23_before_sleep', 'medication']
outcome_variable = 'average_heart_rate'

# Run the function
selected_predictors, model_summaries = add_predictors_by_correlation(df, predictors, outcome_variable)

# Creating DataFrame from the list of model summaries
model_results_df = pd.DataFrame(model_summaries)

coefficients = model_results_df['Coefficients'].apply(pd.Series)
coefficients = coefficients.fillna('')  # Replace NaN with empty strings for readability

# Concatenate the coefficients to the readable DataFrame
model_results_df = pd.concat([model_results_df, coefficients], axis=1)
# drop the coefficients column
model_results_df = model_results_df.drop('Coefficients', axis=1)
# Display the formatted DataFrame
print(model_results_df)

In [ ]:
def add_model_to_results(data, predictors, outcome, results_list):
    formula = "{} ~ {}".format(outcome, ' + '.join(predictors))
    model = smf.ols(formula, data=data).fit()
    model_summary = {
        'Model': ' + '.join(predictors),
        'AIC': model.aic,
        'BIC': model.bic,
        'R-squared': model.rsquared,
        'Adj. R-squared': model.rsquared_adj,
        'Coefficients': model.params.to_dict()
    }
    results_list.append(model_summary)

def stepwise_selection(X, y, initial_list=[], threshold_in=0.05, threshold_out=0.1):
    included = list(initial_list)
    results_list = []  # List to store model summaries
    while True:
        changed = False
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            formula = "{} ~ {}".format(y, ' + '.join(included + [new_column]))
            model = ols(formula, data=df).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            add_model_to_results(df, included, y, results_list)
        
        
        # Backward step
        model = ols("{} ~ {}".format(y, ' + '.join(included)), data=df).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            add_model_to_results(df, included, y, results_list)
        
        if not changed:
            break
    return included, results_list

In [ ]:
# Running the stepwise selection
predictors = ['smart_alarm', 'drinks', 'fastfood', 'sports', 'food_23_before_sleep', 'medication']
outcome_variable = 'average_heart_rate'

selected_predictors, model_summaries = stepwise_selection(df[predictors], outcome_variable)

# Creating DataFrame from the list of model summaries
model_results_df = pd.DataFrame(model_summaries)

# Print the results
print(model_results_df)

In [ ]:
X = data[['Smart alarm', 'Drinks', 'Fastfood', 'Sports', 'Food 2h before sleep', 
          'Medication', 'Woke up by (smart) alarm', 'Woke up by external factors', 'Slept again after alarm', 
          #'time_of_awakening', 'state_before_awakening'
          ]]
y = data['Average First Thirty min']

# Adding a constant to the model (for the intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# View the summary of the model
print(model.summary())


In [ ]:
# Drop rows where y is missing
data_clean = data.dropna(subset=['Average First Ten min'])
X = data_clean[['Smart alarm', 'Drinks', 'Fastfood', 'Sports', 'Food 2h before sleep', 
          'Medication', 'Woke up by (smart) alarm', 'Woke up by external factors', 'Slept again after alarm', 
          #'time_of_awakening', 'state_before_awakening'
          ]]
y = data_clean['Average First Ten min']

# Adding a constant to the model (for the intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# View the summary of the model
print(model.summary())


In [ ]:
X = data[['Smart alarm', 'Drinks', 'Fastfood', 'Sports', 'Food 2h before sleep', 
          'Medication', 'Woke up by (smart) alarm', 'Woke up by external factors', 'Slept again after alarm', 
          #'time_of_awakening', 'state_before_awakening'
          ]]
y = data['Average Lowest Three obs']

# Adding a constant to the model (for the intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# View the summary of the model
print(model.summary())
